In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['SPF']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('specific pathogen free', 172),
 ('sun protection factor', 28),
 ('s phase fraction', 26),
 ('supernatant protein factor', 15),
 ('s phase promoting factor', 7),
 ('sodefrin precursor like factor', 5),
 ('subcapsular proliferative foci', 4),
 ('sweet potato flour', 4),
 ('shortest path function', 3),
 ('sustained pain free', 3),
 ('spiropolyfluorene', 3),
 ('specific pathogen free mice', 3)]

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-03 04:11:48] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'s phase fraction': 'NCIT:C18456',
  's phase promoting factor': 's_phase_promoting_factor',
  'shortest path function': 'ungrounded',
  'sodefrin precursor like factor': 'ungrounded',
  'specific pathogen free': 'MESH:D013047',
  'specific pathogen free mice': 'MESH:D013047',
  'spiropolyfluorene': 'ungrounded',
  'subcapsular proliferative foci': 'ungrounded',
  'sun protection factor': 'MESH:D059405',
  'supernatant protein factor': 'HGNC:10699',
  'sustained pain free': 'MESH:C121034',
  'sweet potato flour': 'ungrounded'},
 {'NCIT:C18456': 'S-Phase Fraction',
  's_phase_promoting_factor': 's_phase_promoting_factor',
  'MESH:D013047': 'Specific Pathogen-Free Organisms',
  'MESH:D059405': 'Sun Protection Factor',
  'HGNC:10699': 'SEC14L2',
  'MESH:C121034': 'Pain-Free'},
 ['MESH:D013047', 'MESH:D059405', 'NCIT:C18456']]

In [19]:
grounding_map, names, pos_labels = [{'s phase fraction': 'NCIT:C18456',
  's phase promoting factor': 's_phase_promoting_factor',
  'shortest path function': 'ungrounded',
  'sodefrin precursor like factor': 'ungrounded',
  'specific pathogen free': 'MESH:D013047',
  'specific pathogen free mice': 'MESH:D013047',
  'spiropolyfluorene': 'ungrounded',
  'subcapsular proliferative foci': 'ungrounded',
  'sun protection factor': 'MESH:D059405',
  'supernatant protein factor': 'HGNC:10699',
  'sustained pain free': 'MESH:C121034',
  'sweet potato flour': 'ungrounded'},
 {'NCIT:C18456': 'S-Phase Fraction',
  's_phase_promoting_factor': 's_phase_promoting_factor',
  'MESH:D013047': 'Specific Pathogen-Free Organisms',
  'MESH:D059405': 'Sun Protection Factor',
  'HGNC:10699': 'SEC14L2',
  'MESH:C121034': 'Pain-Free'},
 ['MESH:D013047', 'MESH:D059405', 'HGNC:10699', 'NCIT:C18456']]

In [10]:
excluded_longforms = []

In [11]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [12]:
additional_entities = {}

In [13]:
unambiguous_agent_texts = {}

In [14]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [15]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [16]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [17]:
intersection1

[('HGNC:17981', 'HGNC:17981', 29)]

In [18]:
intersection2

[('ungrounded', 'HGNC:17981', 0)]

In [20]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['RTCA', 'RTCD1', 'RPC', 'RTC1', 'RTC'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [ ]:
names.update(additional_entitie)

In [20]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-03 04:24:03] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-03 04:24:07] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9262241003417474 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [16]:
classifier.stats

{'label_distribution': {'HGNC:10699': 15,
  'MESH:D013047': 127,
  'NCIT:C18456': 23,
  's_phase_promoting_factor': 4,
  'MESH:D059405': 20,
  'MESH:C121034': 2,
  'ungrounded': 7},
 'f1': {'mean': 0.926773, 'std': 0.054422},
 'precision': {'mean': 0.918578, 'std': 0.03723},
 'recall': {'mean': 0.958824, 'std': 0.044019},
 's_phase_promoting_factor': {'f1': {'mean': 0.8, 'std': 0.4},
  'pr': {'mean': 0.8, 'std': 0.4},
  'rc': {'mean': 0.8, 'std': 0.4}},
 'MESH:C121034': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'NCIT:C18456': {'f1': {'mean': 0.815873, 'std': 0.248066},
  'pr': {'mean': 0.75, 'std': 0.293258},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:D059405': {'f1': {'mean': 0.971429, 'std': 0.057143},
  'pr': {'mean': 0.95, 'std': 0.1},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D013047'

In [21]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [23]:
disamb.dump(model_name, results_path)

In [22]:
print(disamb.info())

Disambiguation model for SPF

Produces the disambiguations:
	Pain-Free	MESH:C121034
	S-Phase Fraction*	NCIT:C18456
	SEC14L2*	HGNC:10699
	Specific Pathogen-Free Organisms*	MESH:D013047
	Sun Protection Factor*	MESH:D059405
	s_phase_promoting_factor	s_phase_promoting_factor

Class level metrics:
--------------------
Grounding                       	Count	F1     
Specific Pathogen-Free Organisms*	127	 0.9414
                S-Phase Fraction*	 23	0.81587
           Sun Protection Factor*	 20	0.97143
                         SEC14L2*	 15	   0.92
                      Ungrounded	  7	    0.0
        s_phase_promoting_factor	  4	    0.8
                       Pain-Free	  2	    0.4

Weighted Metrics:
-----------------
	F1 score:	0.92622
	Precision:	0.92518
	Recall:		0.95135

* Positive labels
See Docstring for explanation



In [24]:
model_to_s3(disamb)